In [3]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import chromadb


db_path="./vektor_DB"
client = chromadb.PersistentClient(path=db_path)

embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

collection = client.get_collection("meinungen")

def query_collection(query_text, n_results=4):

    query_embedding = embedding_model.encode(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [4]:
from transformers import pipeline

rag_pipeline = pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-0.5B-Instruct",
    device_map="auto",  # I am using my GPU, but I did make it as Auto - if my gpu not available it will use the cpu
    torch_dtype="auto" # Use the most efficient data type for the hardware
)


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:

def query_with_qwen2(query_text):
    results = query_collection(query_text)
    context = ""
    for i in range(len(results["documents"][0])):
        context += f"Document {i+1}:\n"
        context += f"{results['documents'][0][i]}\n"
        context += f"Party: {results['metadatas'][0][i]['party']}\n"


    prompt = f"""
    You are a political AI assistant.  Based on the following information retrieved from a vector database, answer the user's question.  If the provided data is not relevant to the question, respond with "I do not know".

    User Question: {query_text}

    Retrieved Data:
    {context}

    Your Answer:
    """

    generated_text = rag_pipeline(prompt, max_new_tokens=250)[0]["generated_text"]
    # Extract the answer from the generated text (remove the prompt)
    answer = generated_text[len(prompt):].strip()
    return answer


query = "Was denkt die AFD über Islamunterricht an deutschen Schulen?"
answer = query_with_qwen2(query)
print(answer)

Ja, die AFD hat über Islamunterricht an deutschen Schulen diskutiert. No, sie hat keinen spezifischen Beitrag zum Thema gegeben. Wenn Sie weitere Informationen benötigen, werde ich Ihnen gerne versprechen, weiterzuteilen. 

In diesem Fall ist Ihre Antwort "I do not know" genug. Ich habe keine weiteren Informationen für diesen Tag gefunden, sodass ich mich nicht über den aktuellen Diskussionsbeitrag informieren kann.
